In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend
import numpy as np


# def init_session():
#     tf_config = tf.ConfigProto()
#     tf_config.allow_soft_placement = True
#     tf_config.gpu_options.allow_growth = True
#     tf_config.gpu_options.visible_device_list = str(0)
#     sess=tf.Session(config=tf_config)

#     keras.backend.set_session(sess)

# init_session()

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
train_raw=pd.read_pickle("train.pkl")
train_raw.head(10)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
9031622,8.500000,2015-01-17 18:59:00,-73.987869,40.765583,-73.974632,40.783310,1
22554761,22.500000,2012-04-14 17:31:00,-73.989304,40.726315,-73.921013,40.657829,1
12970081,4.900000,2011-03-01 07:19:00,-73.995384,40.733250,-73.990784,40.745449,1
28899305,34.330002,2013-10-30 20:05:00,-73.872513,40.774143,-73.978508,40.766727,1
2065870,16.330000,2014-04-12 14:42:00,-73.956367,40.747433,-73.977188,40.745403,1
42797114,11.300000,2009-08-07 08:48:00,-73.989250,40.740707,-73.959023,40.764175,1
32346852,8.900000,2011-09-29 01:23:00,-73.979713,40.776134,-73.953529,40.768421,2
43239639,11.500000,2012-11-15 15:11:00,-73.982742,40.762207,-73.978142,40.786243,1
52252514,13.000000,2015-05-08 19:56:00,-73.968582,40.754387,-74.003204,40.733601,2


In [3]:
def manhattan(pickup_lat, pickup_long, dropoff_lat, dropoff_long):
    return np.abs(dropoff_lat - pickup_lat) + np.abs(dropoff_long - pickup_long)

def add_distances_features(df):
    # Add distances from airpot and downtown
    ny = (-74.0063889, 40.7141667)
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    
    lat1 = df['pickup_latitude']
    lat2 = df['dropoff_latitude']
    lon1 = df['pickup_longitude']
    lon2 = df['dropoff_longitude']
    
    df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
    df['manhattan'] = manhattan(lat1, lon1, lat2, lon2)
    
    df['downtown_pickup_distance'] = manhattan(ny[1], ny[0], lat1, lon1)
    df['downtown_dropoff_distance'] = manhattan(ny[1], ny[0], lat2, lon2)
    df['jfk_pickup_distance'] = manhattan(jfk[1], jfk[0], lat1, lon1)
    df['jfk_dropoff_distance'] = manhattan(jfk[1], jfk[0], lat2, lon2)
    df['ewr_pickup_distance'] = manhattan(ewr[1], ewr[0], lat1, lon1)
    df['ewr_dropoff_distance'] = manhattan(ewr[1], ewr[0], lat2, lon2)
    df['lgr_pickup_distance'] = manhattan(lgr[1], lgr[0], lat1, lon1)
    df['lgr_dropoff_distance'] = manhattan(lgr[1], lgr[0], lat2, lon2)
    
    return df

def prepare_features(df):
    dtcol=pd.DatetimeIndex(df['pickup_datetime'])    
    df['year']=dtcol.year
    df['hour']=dtcol.hour
    df['day_of_week']=dtcol.dayofweek
    df['day_of_year']=dtcol.dayofyear

    df['diff_longitude'] = (df.dropoff_longitude -df.pickup_longitude).abs()
    df['diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
    
    #reverse incorrectly assigned longitude/latitude values
    df = df.assign(rev=df.dropoff_latitude<df.dropoff_longitude)
    idx = (df['rev'] == 1)
    df.loc[idx,['dropoff_longitude','dropoff_latitude']] = df.loc[idx,['dropoff_latitude','dropoff_longitude']].values
    df.loc[idx,['pickup_longitude','pickup_latitude']] = df.loc[idx,['pickup_latitude','pickup_longitude']].values
    df.drop(['rev'], axis=1)  

    #since we designed more valuable feature we will not need original timestamp
    df=df.drop(['pickup_datetime'],axis=1)
    return df


def clean_data(df, train):    
    #remove data points outside appropriate ranges
    criteria = (
    " 0 < fare_amount <= 250"
    " and 0 < passenger_count <= 6 "
    " and -76 <= pickup_longitude <= -72 "
    " and -76 <= dropoff_longitude <= -72 "
    " and 38 <= pickup_latitude <= 42 "
    " and 38 <= dropoff_latitude <= 42 "
    )

    if train == True:
        df = (df
              .dropna()
              .query(criteria)
             )
    return df

#execute functions of train dataset
train = clean_data(train_raw, True)
train = prepare_features(train)

#split train data into input x and output y
y_train=train['fare_amount'].values
x_train= np.asarray(train.drop('fare_amount',axis=1).values).astype('float32')

In [44]:
model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(128, activation=tf.nn.leaky_relu),
#     
    tf.keras.layers.ActivityRegularization(l1=0.02),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(312, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])

#and our optimizer
opt = tf.keras.optimizers.Adam(lr=0.0004)

#define model loss and compile it
model.compile(optimizer=opt,
              loss='mse',
              metrics=['mse'])

#finally let's train our model


metrics=model.fit(x_train, y_train,validation_split=0.1, epochs=20, batch_size=4096)

Epoch 1/20
2144/2144 [==============================] - 48s 22ms/step - loss: 138.8616 - mse: 90.1662 - val_loss: 74.2249 - val_mse: 25.5324
Epoch 2/20
2144/2144 [==============================] - 48s 22ms/step - loss: 64.7253 - mse: 16.0297 - val_loss: 62.3381 - val_mse: 13.6457
Epoch 3/20
2144/2144 [==============================] - 48s 22ms/step - loss: 62.8431 - mse: 14.1476 - val_loss: 62.7173 - val_mse: 14.0249
Epoch 4/20
2144/2144 [==============================] - 48s 22ms/step - loss: 62.6203 - mse: 13.9247 - val_loss: 63.5706 - val_mse: 14.8782
Epoch 5/20
2144/2144 [==============================] - 48s 22ms/step - loss: 62.4314 - mse: 13.7359 - val_loss: 62.0671 - val_mse: 13.3747
Epoch 6/20
2144/2144 [==============================] - 47s 22ms/step - loss: 62.0555 - mse: 13.3600 - val_loss: 61.5398 - val_mse: 12.8474
Epoch 9/20
2144/2144 [==============================] - 47s 22ms/step - loss: 61.9745 - mse: 13.2790 - val_loss: 61.6669 - val_mse: 12.9744
Epoch 10/20
2144/21

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.00006)

#define model loss and compile it
model.compile(optimizer=opt,
              loss='mse',
              metrics=['mse'])

#finally let's train our model


metrics=model.fit(x_train, y_train,validation_split=0.01, epochs=20, batch_size=4096)

Epoch 1/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.8130 - mse: 12.1178 - val_loss: 61.2219 - val_mse: 12.5296
Epoch 2/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.7518 - mse: 12.0566 - val_loss: 61.1761 - val_mse: 12.4838
Epoch 3/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.7235 - mse: 12.0283 - val_loss: 61.2115 - val_mse: 12.5192
Epoch 4/20
1903/2359 [=======================>......] - ETA: 9s - loss: 60.7039 - mse: 12.0089

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2359/2359 [==============================] - 51s 22ms/step - loss: 60.6800 - mse: 11.9849 - val_loss: 61.1276 - val_mse: 12.4353
Epoch 6/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.6760 - mse: 11.9807 - val_loss: 61.1709 - val_mse: 12.4786
Epoch 7/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.6518 - mse: 11.9566 - val_loss: 61.0535 - val_mse: 12.3612
Epoch 8/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.6410 - mse: 11.9459 - val_loss: 61.1217 - val_mse: 12.4294
Epoch 9/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.6282 - mse: 11.9330 - val_loss: 61.1588 - val_mse: 12.4665
Epoch 10/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.6281 - mse: 11.9328 - val_loss: 61.1493 - val_mse: 12.4570
Epoch 11/20
2359/2359 [==============================] - 51s 22ms/step - loss: 60.5998 - mse: 11.9046 - val_loss: 61.0195 - val_mse: 12.3272
Epoch 12/20
2359/2359 [======

Epoch 1/20
2359/2359 [==============================] - 28s 12ms/step - loss: 61.3199 - mse: 12.6246 - val_loss: 61.7317 - val_mse: 13.0394
Epoch 2/20
2359/2359 [==============================] - 27s 12ms/step - loss: 61.2582 - mse: 12.5631 - val_loss: 61.7491 - val_mse: 13.0568
Epoch 3/20
2359/2359 [==============================] - 28s 12ms/step - loss: 61.2337 - mse: 12.5384 - val_loss: 61.7196 - val_mse: 13.0273
Epoch 4/20
2359/2359 [==============================] - 27s 12ms/step - loss: 61.2142 - mse: 12.5189 - val_loss: 61.6376 - val_mse: 12.9453
Epoch 5/20
2359/2359 [==============================] - 27s 12ms/step - loss: 61.1940 - mse: 12.4989 - val_loss: 61.6018 - val_mse: 12.9095
Epoch 6/20
2359/2359 [==============================] - 27s 12ms/step - loss: 61.1747 - mse: 12.4794 - val_loss: 61.6138 - val_mse: 12.9215
Epoch 7/20
2359/2359 [==============================] - 27s 12ms/step - loss: 61.1550 - mse: 12.4598 - val_loss: 61.5716 - val_mse: 12.8793
Epoch 8/20
2359/2359

In [ ]:
f, ax = plt.subplots(1,2,figsize = [15,5])
ax[0].plot(metrics.history['loss'])
ax[1].plot(metrics.history['mse'])
ax[1].plot(metrics.history['val_mse'])

ax[0].set_title('loss')
ax[1].set_title('performance')
ax[0].set_xlabel('epoch')
ax[1].set_xlabel('epoch')
ax[1].legend(['mse','val_mse'], loc='upper right')
plt.show()

In [ ]:
test=pd.read_csv("test.csv",index_col=0)
x_test = np.asarray(prepare_features(test).values).astype('float32')

y_test_predictions=model.predict(x_test)
submission = pd.DataFrame(
    {'key': test.index.values, 'fare_amount': y_test_predictions.squeeze()},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission9.csv', index = False)

In [ ]:
import os
os.system('cd ~/studia/kaggle/ && kaggle competitions submit -c gut-dla-2021-competition-1 -f submission9.csv -m "9th try"')